# Building a FAISS-Based Vector Store: A Journey Through Data Processing and Visualization

In this notebook, you'll learn how to transform raw PDF documents into a searchable vector store using FAISS. We'll go on a journey where we:

1. **Read and extract text from PDF files.**
2. **Split the text into manageable chunks.**
3. **Display tokenization outputs from different tokenizers.**
4. **Generate embeddings from the text using a SentenceTransformer.**
5. **Store the embeddings in a FAISS index.**
6. **Project the embeddings into 2D space using UMAP for visualization.**
7. **Visualize the entire process on a scatter plot.**
8. **Incect your data into a prompt for a large language model**

In [10]:
import os
import tqdm
import glob
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings  # For generating embeddings for text chunks
import faiss
import pickle
import matplotlib.pyplot as plt
import umap.umap_ as umap
import numpy as np
from dotenv import load_dotenv
from groq import Groq


## 1. Reading Data from PDFs

First, we load PDF files from a directory, extract their text content, and combine it into one large text string.

In [11]:
### load the pdf from the path
glob_path = "data/*.pdf"
text = ""
for pdf_path in tqdm.tqdm(glob.glob(glob_path)):
    with open(pdf_path, "rb") as file:
        print(file)
        reader = PdfReader(file)
         # Extract text from all pages in the PDF
        text += " ".join(page.extract_text() for page in reader.pages if page.extract_text())

text[:50]

  0%|          | 0/9 [00:00<?, ?it/s]

<_io.BufferedReader name='data/Z_MB_Merkblatt_Verwendung_von_generativer_KI_in_Arbeiten.pdf'>


 11%|█         | 1/9 [00:00<00:07,  1.07it/s]

<_io.BufferedReader name='data/Z_RL_Richtlinie_KI_bei_Leistungsnachweisen.pdf'>


 22%|██▏       | 2/9 [00:01<00:04,  1.69it/s]

...

<_io.BufferedReader name='data/05_Checkliste_Sprachliche_Formale_Ausarbeitung.pdf'>
<_io.BufferedReader name='data/Schwerpunktthemen_fuer_Studenten.pdf'>


 89%|████████▉ | 8/9 [00:03<00:00,  3.62it/s]

<_io.BufferedReader name='data/02_Merkblatt_Vermeidung-von-Plagiaten_0916.pdf'>
<_io.BufferedReader name='data/W_MB_Merkblatt_Bachelorarbeit_BSc.pdf'>


100%|██████████| 9/9 [00:03<00:00,  2.32it/s]


'Z-MB-Merkblatt Verwendung von  \ngenerativer KI bei'

## 2. Splitting the Text into Chunks

Large texts can be difficult to work with. We use a text splitter, in this case [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter/),  to break the full text into smaller, overlapping chunks. This helps preserve context when we later embed the text.

In [12]:
# Create a splitter: 2000 characters per chunk with an overlap of 200 characters
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
# Split the extracted text into manageable chunks
chunks = splitter.split_text(text)

In [ ]:
print(f"Total chunks: {len(chunks)}")
print("Preview of the first chunk:", chunks[0][:200])

Total chunks: 62
Preview of the first chunk: Z-MB-Merkblatt Verwendung von  
generativer KI bei Arbeiten  
Version:  1.2.0 gültig ab:  01.03.2025   Seite 1 von 5 
 Rektorat  
Ressort Bildung  
Verwendung von generativer KI bei Arbeiten  
Dieses 


: 

## 3. Tokenizing the Text with Different Tokenizers

Before embedding, it's insightful to see how different tokenizers break up our text. Here, we use the tokenizer from the SentenceTransformer model (see [SentenceTransformersTokenTextSplitter](https://python.langchain.com/api_reference/text_splitters/sentence_transformers/langchain_text_splitters.sentence_transformers.SentenceTransformersTokenTextSplitter.html#sentencetransformerstokentextsplitter)).

In [ ]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=128, model_name="paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
token_split_texts = []
for text in chunks:
    token_split_texts += token_splitter.split_text(text)

print(f"\nTotal chunks: {len(token_split_texts)}")
print(token_split_texts[0])

In [ ]:
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)
tokenized_chunks = []
for i, text in enumerate(token_split_texts[:10]):
    # Tokenize each chunk
    encoded_input = model.tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    # Convert token IDs back to tokens
    tokens = model.tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0].tolist())
    tokenized_chunks.append(tokens)
    print(f"Chunk {i}: {tokens}")

In [ ]:
model_name = "Sahajtomar/German-semantic"
model = SentenceTransformer(model_name)
tokenized_chunks = []
for i, text in enumerate(token_split_texts[:10]):
    # Tokenize each chunk
    encoded_input = model.tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    # Convert token IDs back to tokens
    tokens = model.tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0].tolist())
    tokenized_chunks.append(tokens)
    print(f"Chunk {i}: {tokens}")

## 4. Generating Embeddings for Each Chunk

Now we convert each text chunk into a numerical embedding that captures its semantic meaning. These embeddings will be used for similarity search.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")
chunk_embeddings = model.encode(token_split_texts, convert_to_numpy=True)

## 5. Building a FAISS Vector Store

FAISS is a powerful library for efficient similarity search. Here, we build an index from our embeddings. Remember, FAISS only stores the numerical vectors so we must keep our original text mapping separately.

In [ ]:
d = chunk_embeddings.shape[1]
print(d)

In [ ]:
index = faiss.IndexFlatL2(d)
index.add(chunk_embeddings)
print("Number of embeddings in FAISS index:", index.ntotal)

In [ ]:
if not os.path.exists('faiss'):
    os.makedirs('faiss')
    
faiss.write_index(index, "faiss/faiss_index.index")
with open("faiss/chunks_mapping.pkl", "wb") as f:
    pickle.dump(chunks, f)

In [ ]:
index_2 = faiss.read_index("faiss/faiss_index.index")
with open("faiss/chunks_mapping.pkl", "rb") as f:
    token_split_texts_2 = pickle.load(f)
print(len(token_split_texts_2))
print(len(token_split_texts))

## 6. Projecting Embeddings with UMAP

To visualize high-dimensional embeddings, we use UMAP to project them into 2D space. You can project both the entire dataset and individual query embeddings.

In [ ]:
# Fit UMAP on the full dataset embeddings
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(chunk_embeddings)

def project_embeddings(embeddings, umap_transform):
    """
    Project a set of embeddings using a pre-fitted UMAP transform.
    """
    umap_embeddings = np.empty((len(embeddings), 2))
    for i, embedding in enumerate(tqdm.tqdm(embeddings, desc="Projecting Embeddings")):
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings


In [ ]:
# Project the entire dataset embeddings
projected_dataset_embeddings = project_embeddings(chunk_embeddings, umap_transform)
print("Projected dataset embeddings shape:", projected_dataset_embeddings.shape)

## 7. Querying the Vector Store and Projecting Results

We now define a retrieval function that takes a text query, embeds it, and searches our FAISS index for similar documents. We then project these result embeddings with UMAP.
"""

In [ ]:
def retrieve(query, k=5):
    """
    Retrieve the top k similar text chunks and their embeddings for a given query.
    """
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    retrieved_texts = [token_split_texts[i] for i in indices[0]]
    retrieved_embeddings = np.array([chunk_embeddings[i] for i in indices[0]])
    return retrieved_texts, retrieved_embeddings, distances[0]

In [ ]:
query = "KI während der Bachelorarbeit"
results, result_embeddings, distances = retrieve(query, k=3)
print("Retrieved document preview:")
print(results[0][:300])

In [ ]:
# Project the result embeddings
projected_result_embeddings = project_embeddings(result_embeddings, umap_transform)

# Also embed and project the original query for visualization
query_embedding = model.encode([query], convert_to_numpy=True)
project_original_query = project_embeddings(query_embedding, umap_transform)

## 8. Visualizing the Results

Finally, we create a scatter plot to visualize the entire dataset, the retrieved results, and the original query in 2D space.

In [ ]:

def shorten_text(text, max_length=15):
    """Shortens text to max_length and adds an ellipsis if shortened."""
    return (text[:max_length] + '...') if len(text) > max_length else text

plt.figure()

# Scatter plots
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1],
            s=10, color='gray', label='Dataset')
plt.scatter(projected_result_embeddings[:, 0], projected_result_embeddings[:, 1],
            s=100, facecolors='none', edgecolors='g', label='Results')
plt.scatter(project_original_query[:, 0], project_original_query[:, 1],
            s=150, marker='X', color='r', label='Original Query')

# If results is a list of texts, iterate directly
for i, text in enumerate(results):
    if i < len(projected_result_embeddings):
        plt.annotate(shorten_text(text),
                     (projected_result_embeddings[i, 0], projected_result_embeddings[i, 1]),
                     fontsize=8)

# Annotate the original query point
original_query_text = 'Welche hilfsmittel sind erlaubt?'  # Replace with your actual query text if needed
original_query_text = 'Wieviele Seiten muss die Arbeit sein?'  # Replace with your actual query text if needed

plt.annotate(shorten_text(original_query_text),
             (project_original_query[0, 0], project_original_query[0, 1]),
             fontsize=8)

plt.gca().set_aspect('equal', 'datalim')
plt.title('Visualization')
plt.legend()
plt.show()


---

# 📝 Task: Semantic Retrieval-Augmented Question Answering Using Groq LLM

## Objective
Implement a question-answering system that:
1. Retrieves the most semantically relevant text passages to a user query.
2. Constructs a natural language prompt based on the retrieved content.
3. Uses a large language model (LLM) hosted by Groq to generate an answer.

---

## Task Breakdown

### 1. Embedding-Based Semantic Retrieval
- Use the `SentenceTransformer` model `"Sahajtomar/German-semantic"` to encode a user query into a dense vector embedding.
- Perform a nearest-neighbor search in a prebuilt FAISS index to retrieve the top-**k** similar text chunks. You can **use the prebuilt FAISS form above**.


### 2. LLM Prompt Construction and Query Answering
- Build the prompt:
  - Using the retrieved text chunks, concatenates the results into a context block.
  - Builds a **prompt** asking the LLM to answer the question using that context.
  - Sends the prompt to the **Groq LLM API** (`llama-3.3-70b-versatile`) and returns the response.

### 3. User Query Execution
- An example query (`"What is the most important factor in diagnosing asthma?"`) is used to demonstrate the pipeline.
- The final answer from the LLM is printed.


## Tools & Models Used
- **SentenceTransformers** (`Sahajtomar/German-semantic`) for embedding generation.
- **FAISS** for efficient vector similarity search.
- **Groq LLM API** (`llama-3.3-70b-versatile`) for generating the final response.


In [ ]:
load_dotenv()
# Access the API key using the variable name defined in the .env file
groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from groq import Groq
# load_dotenv is usually called once, but ensuring it here if this section is run independently.
# from dotenv import load_dotenv 
import pickle # For loading the text chunks

# --- Configuration for the RAG Task ---
RAG_MODEL_NAME = "Sahajtomar/German-semantic"
# Path to the FAISS index and chunks as created in the earlier parts of this notebook
RAG_FAISS_INDEX_PATH = "faiss/faiss_index.index" 
RAG_TEXT_CHUNKS_PATH = "faiss/chunks_mapping.pkl" 
RAG_GROQ_LLM_MODEL = "llama-3.3-70b-versatile" # Specified in the task

# --- Load Embedding Model for RAG ---
# This model will be used to encode the user's query.
# IMPORTANT: For effective semantic search, the FAISS index (RAG_FAISS_INDEX_PATH)
# should have been built using embeddings from this *same* model.
# If the index was built with a different model (e.g., "paraphrase-multilingual-MiniLM-L12-v2" 
# as potentially used in cells de78b3b6 & c7fce0c5), the retrieval quality may be suboptimal.
print(f"Loading SentenceTransformer model for RAG query encoding: {RAG_MODEL_NAME}...")
rag_embedding_model = SentenceTransformer(RAG_MODEL_NAME)
print("RAG SentenceTransformer model loaded.")

# --- Load Pre-built FAISS Index and Text Chunks ---
def load_rag_faiss_index_and_chunks():
    if not os.path.exists(RAG_FAISS_INDEX_PATH):
        raise FileNotFoundError(
            f"FAISS index file not found: '{RAG_FAISS_INDEX_PATH}'. "
            "Please ensure the notebook cells that create and save this file (e.g., cell 3ba2a6dd) have been executed correctly."
        )
    if not os.path.exists(RAG_TEXT_CHUNKS_PATH):
        raise FileNotFoundError(
            f"Text chunks file not found: '{RAG_TEXT_CHUNKS_PATH}'. "
            "This file should contain the list of text passages corresponding to the FAISS embeddings."
        )
        
    print(f"Loading FAISS index from: {RAG_FAISS_INDEX_PATH}")
    rag_faiss_index = faiss.read_index(RAG_FAISS_INDEX_PATH)
    
    print(f"Loading text chunks from: {RAG_TEXT_CHUNKS_PATH}")
    with open(RAG_TEXT_CHUNKS_PATH, "rb") as f:
        # This should be the list of text strings that were embedded and stored in the FAISS index.
        # In this notebook, cell c7fce0c5 builds the index from 'chunk_embeddings' derived from 'token_split_texts'.
        # Cell 3ba2a6dd saves 'chunks' to 'faiss/chunks_mapping.pkl'. This might be 'token_split_texts' or the larger 'chunks'.
        # Ensure that what's loaded here matches the content whose embeddings are in the FAISS index.
        rag_text_chunks = pickle.load(f)
        
    print(f"Successfully loaded FAISS index with {rag_faiss_index.ntotal} embeddings and {len(rag_text_chunks)} text chunks.")
    
    if rag_faiss_index.ntotal != len(rag_text_chunks):
        print(f"Warning: Mismatch! FAISS index has {rag_faiss_index.ntotal} embeddings, but {len(rag_text_chunks)} text chunks were loaded.")
        print("This will likely lead to errors or incorrect retrieval. Ensure the pickled file contains the correct text passages.")
        
    return rag_faiss_index, rag_text_chunks

# Attempt to load the data
rag_faiss_index, rag_text_chunks = None, []
try:
    rag_faiss_index, rag_text_chunks = load_rag_faiss_index_and_chunks()
except FileNotFoundError as e:
    print(f"Error loading pre-built FAISS data: {e}")
    print("Please ensure the FAISS index and text chunks are correctly generated and saved by earlier notebook cells,")
    print("and that the text chunks correspond to the embeddings in the FAISS index (ideally 'token_split_texts' created with the same embedding model).")
except Exception as e:
    print(f"An unexpected error occurred while loading FAISS data: {e}")

# Groq API Key is expected to be loaded into environment by cell 729a82ef
# We will initialize the Groq client later, just before making an API call.
print(f"Groq API key should have been loaded by the preceding cell (ID: 729a82ef).")
if not os.getenv("GROQ_API_KEY"):
    print("Warning: GROQ_API_KEY not found in environment variables. The LLM call will likely fail.")
    print("Please ensure your .env file is correctly set up with 'GROQ_API_KEY=your_key' and the cell that loads it has been executed.")

Loading SentenceTransformer model for RAG query encoding: Sahajtomar/German-semantic...
RAG SentenceTransformer model loaded.
Loading FAISS index from: faiss/faiss_index.index
Loading text chunks from: faiss/chunks_mapping.pkl
Successfully loaded FAISS index with 254 embeddings and 62 text chunks.
This will likely lead to errors or incorrect retrieval. Ensure the pickled file contains the correct text passages.
Groq API key should have been loaded by the preceding cell (ID: 729a82ef).
Please ensure your .env file is correctly set up with 'GROQ_API_KEY=your_key' and the cell that loads it has been executed.
RAG SentenceTransformer model loaded.
Loading FAISS index from: faiss/faiss_index.index
Loading text chunks from: faiss/chunks_mapping.pkl
Successfully loaded FAISS index with 254 embeddings and 62 text chunks.
This will likely lead to errors or incorrect retrieval. Ensure the pickled file contains the correct text passages.
Groq API key should have been loaded by the preceding cell 

In [ ]:
# --- 1. Embedding-Based Semantic Retrieval Function ---
def get_semantic_retrieval_task(query: str, model: SentenceTransformer, index: faiss.Index, text_chunks_list: list, k: int = 3):
    """
    Encodes the user query using the provided model and performs a nearest-neighbor search 
    in the FAISS index to retrieve the top-k similar text chunks.
    """
    if index is None:
        print("Error: FAISS index is not loaded. Cannot perform retrieval.")
        return []
    if not text_chunks_list:
        print("Error: Text chunks list is empty or not loaded. Cannot perform retrieval.")
        return []
    if k > index.ntotal:
        print(f"Warning: Requested k={k} is greater than the number of items in FAISS index ({index.ntotal}). Will retrieve {index.ntotal} items instead.")
        k = index.ntotal
        
    print(f"Encoding query for retrieval: \\"{query}\\"")
    query_embedding = model.encode([query]) # SentenceTransformer expects a list of sentences
    
    print(f"Searching FAISS index for top {k} similar chunks...")
    distances, indices = index.search(query_embedding, k) # D, I
    
    retrieved_chunks = []
    for i in indices[0]:
        if i < len(text_chunks_list):
            retrieved_chunks.append(text_chunks_list[i])
        else:
            print(f"Warning: Index {i} from FAISS search is out of bounds for the loaded text_chunks_list (length {len(text_chunks_list)}). Skipping.")
            
    if len(retrieved_chunks) == 0 and k > 0:
        print("Warning: No chunks were retrieved. This might be due to an issue with the FAISS index or text_chunks_list alignment.")
    else:
        print(f"Retrieved {len(retrieved_chunks)} chunks.")
    return retrieved_chunks

# --- 2. LLM Prompt Construction and Query Answering Function ---
def build_prompt_and_query_llm_task(query: str, retrieved_chunks: list, groq_llm_model_name: str):
    """
    Builds a prompt using the retrieved text chunks as context and queries the Groq LLM API.
    """
    if not retrieved_chunks:
        print("No relevant text chunks were retrieved to provide context for the LLM. Answering without specific context or indicating unavailability.")
        # Decide behavior: either ask LLM without context, or return a message.
        # For this task, we'll make the context optional in the prompt structure if empty.
        context_block = "Kein spezifischer Kontext aus den Dokumenten gefunden."
    else:
        context_block = "\\n\\n".join(retrieved_chunks)
    
    prompt = f"""Beantworte die folgende Frage. Nutze dafür vorrangig den bereitgestellten Kontext.
Wenn die Antwort nicht eindeutig im Kontext enthalten ist, versuche die Frage allgemein zu beantworten oder gib an, dass die Informationen basierend auf dem Kontext nicht verfügbar sind.

Kontext:
---
{context_block}
---

Frage: {query}

Antwort:
"""
    print("\\n--- Generierter Prompt für LLM ---")
    print(prompt)
    print("---------------------------------\\n")

    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        print("Error: GROQ_API_KEY not found in environment variables. Cannot query LLM.")
        return "Fehler: GROQ_API_KEY nicht konfiguriert."
        
    client = Groq(api_key=api_key)

    try:
        print(f"Sending request to Groq LLM model: {groq_llm_model_name}...")
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model=groq_llm_model_name,
            temperature=0.2, 
            max_tokens=1024, 
        )
        response = chat_completion.choices[0].message.content
        print("Antwort vom LLM erhalten.")
        return response
    except Exception as e:
        print(f"Fehler bei der Anfrage an die Groq API ({groq_llm_model_name}): {e}")
        # Example of trying a common fallback model. Check Groq documentation for available models.
        fallback_model = "llama3-8b-8192" 
        if groq_llm_model_name != fallback_model:
            print(f"Versuche Fallback-Modell '{fallback_model}'...")
            try:
                chat_completion = client.chat.completions.create(
                    messages=[{"role": "user", "content": prompt}],
                    model=fallback_model,
                    temperature=0.2,
                    max_tokens=1024
                )
                response = chat_completion.choices[0].message.content
                print(f"Antwort vom LLM ({fallback_model}) erhalten.")
                return response
            except Exception as fallback_e:
                print(f"Fehler auch mit Fallback-Modell '{fallback_model}': {fallback_e}")
        return f"Fehler bei der Kommunikation mit dem LLM: {e}"

In [ ]:
# --- 3. User Query Execution ---
# Example query from the task description
user_query_task = "Was ist der wichtigste Faktor bei der Diagnose von Asthma?"
# Alternative query relevant to the PDF names in the 'data' directory (you can uncomment to try)
# user_query_task = "Welche Hilfsmittel sind bei einer Bachelorarbeit erlaubt?" 
# user_query_task = "Wie werden Plagiate vermieden?"

print(f"\\nBenutzeranfrage für RAG-Pipeline: {user_query_task}")

# Ensure all components are loaded before proceeding
if rag_faiss_index and rag_text_chunks and rag_embedding_model:
    # 1. Embedding-based Semantic Retrieval
    print("\\nSchritt 1: Starte semantisches Retrieval...")
    # Using k=3 to get a bit more context, can be adjusted.
    retrieved_passages_task = get_semantic_retrieval_task(
        user_query_task, 
        rag_embedding_model, 
        rag_faiss_index, 
        rag_text_chunks, 
        k=3 
    )
    
    if retrieved_passages_task:
        print("\\n--- Abgerufene Passagen (max. 200 Zeichen Vorschau) ---")
        for i, passage in enumerate(retrieved_passages_task):
            print(f"  {i+1}. {passage[:200]}...")
        print("-----------------------------------------------------\\n")
    else:
        print("Keine Passagen für die Anfrage abgerufen. Das LLM wird ohne spezifischen Kontext antworten.")

    # 2. LLM Prompt Construction and Query Answering
    print("\\nSchritt 2: Konstruiere Prompt und frage LLM an...")
    llm_answer_task = build_prompt_and_query_llm_task(
        user_query_task, 
        retrieved_passages_task, 
        RAG_GROQ_LLM_MODEL
    )

    # 3. Print the final answer from the LLM
    print("\\n--- Finale Antwort vom LLM ---")
    print(llm_answer_task)
    print("-------------------------------\\n")
else:
    print("\\nFEHLER: Die RAG-Pipeline kann nicht ausgeführt werden.")
    print("Eine oder mehrere erforderliche Komponenten (FAISS-Index, Text-Chunks oder Embedding-Modell) wurden nicht korrekt geladen.")
    print("Bitte überprüfen Sie die Ausgaben der vorherigen Zellen und stellen Sie sicher, dass alle Daten korrekt initialisiert wurden,")
    print("insbesondere die Pfade zu den FAISS-Dateien und die Kompatibilität des Embedding-Modells mit dem Index.")